In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets

from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id

from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch

from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs
                                                  
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

from tf_lab.nips.shape_net import pc_loader as sn_pc_loader
from tf_lab.autopredictors.exploration import latent_embedding_of_entire_dataset
from tf_lab.point_clouds.raw_gan import RawGAN
from tf_lab.point_clouds.latent_gan import LatentGAN

from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder


from tf_lab.nips.evaluate_gan import entropy_of_occupancy_grid, jensen_shannon_divergence, \
                                     sample_pclouds_distances

from tf_lab.point_clouds.gan import model_saver_id

from tf_lab.autopredictors.exploration import find_neighbors

from tf_lab.nips.helper import pclouds_centered_and_half_sphere

from tf_lab.point_clouds.generators_discriminators import latent_code_discriminator_two_layers, latent_code_generator_two_layers    

from tf_lab.icml.ae_farm_helper import relevant_class

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [18]:
n_pc_samples = 2048
do_training = True
do_plotting = True
ae_loss = 'emd'

save_synthetic_samples = True
save_model = True
saver_step = 10

# class_name = raw_input('Give me the class type: ').lower()
class_name = 'chair'
# ae_id = raw_input('Give me the AE/experiment ID: ').lower()
ae_id = '12' # 128 bottleneck
# cmp_type = raw_input('Comparison Type (best epoch vs. max_min epoch): ').lower()
cmp_type = 'best'

syn_id = shape_net_category_to_synth_id()[class_name]

experiment_name = '_'.join(['nb_gan_ae', ae_id, ae_loss, class_name, str(n_pc_samples), cmp_type, 'epoch'])

top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
synthetic_data_out_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/icml/synthetic_point_clouds'
top_train_dir = osp.join(top_data_dir, 'OUT/icml/nn_models/gan/nb_gan/momentum_search')

In [7]:
# Load Raw Point-Clouds of class
pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_samples))
pclouds_path = osp.join(pclouds_path, syn_id)
file_names = pio.load_filenames_of_input_data(pclouds_path, '.ply')
pclouds, model_ids, syn_ids = pio.load_crude_point_clouds(file_names=file_names, n_threads=50, loader=sn_pc_loader)
print '%d files containing complete point clouds were found.' % (len(pclouds), )

6778 files containing complete point clouds were found.


In [8]:
ae_help = relevant_class(ae_loss)

In [9]:
# Load Auto-Encoder
ae_net_name = 'ae_farm_' + class_name + '_conv_arch_' + ae_id + '_2048pts_' + ae_loss
ae_net_dir = osp.join(top_data_dir, 'OUT/icml/nn_models/ae_farming', ae_net_name)
ae_conf = Conf.load(osp.join(ae_net_dir, 'configuration'))
reset_tf_graph()
ae = PointNetAutoEncoder(ae_net_name, ae_conf)
bneck = ae.bottleneck_size
assert(ae_help.experiment_id_to_bneck[int(ae_id)] == bneck) 

if cmp_type.startswith('best'):
    load_epoch, _ = ae_help.best_epochs[bneck]
    print load_epoch
elif cmp_type.startswith('max'):
    load_epoch, _ = ae_help.max_min_epochs[bneck]
else:
    assert(False)
    saved_epochs = read_saved_epochs(ae_conf.train_dir)
    load_epoch = saved_epochs[-1]

ae.restore_model(ae_conf.train_dir, load_epoch, verbose=True)

990
Model restored in epoch 990.


In [11]:
# Convert raw-data to latent codes.
raw_data = PointCloudDataSet(pclouds, init_shuffle=False)
_, latent_codes, _ = latent_embedding_of_entire_dataset(raw_data, ae, ae_conf)
training_data = PointCloudDataSet(latent_codes)

In [1]:
init_lr = 0.0001
batch_size = 50

noise_params = {'mu':0, 'sigma': 0.5}
noise_dim = ae.bottleneck_size
max_epochs = 500

n_syn_samples = training_data.num_examples  # Same number as original GT data.
n_out = [ae.bottleneck_size]

momentum_grid = [0.9, 0.5, 0.2, 0.1, 0.0, -0.1, -0.2, -0.5, -0.9]
for momentum in momentum_grid:
    train_dir = osp.join(top_train_dir, experiment_name, str(momentum))
    save_dir = osp.join(synthetic_data_out_dir, 'momentum_search', experiment_name, str(momentum))    
    create_dir(train_dir)
    create_dir(save_dir)
    reset_tf_graph()
    gan = LatentGAN(experiment_name, init_lr, n_out, noise_dim, \
                    latent_code_discriminator_two_layers, latent_code_generator_two_layers, \
                    beta=momentum)
    
    
    buf_size = 1 # flush each line
    if do_training:    
        train_stats = []
        log_file = open(osp.join(train_dir, 'train_stats.txt'), 'a', buf_size)
        for _ in range(max_epochs):        
            loss, duration = gan._single_epoch_train(training_data, batch_size, noise_params)
            epoch = int(gan.sess.run(gan.epoch.assign_add(tf.constant(1.0))))
            print epoch, loss                
            log_file.write('%04d\t%.9f\t%.9f\t%.4f\n' % (epoch, loss[0], loss[1], duration / 60.0))        

            if save_model and (epoch % saver_step == 0 or epoch <= 5):
                checkpoint_path = osp.join(train_dir, model_saver_id)
                gan.saver.save(gan.sess, checkpoint_path, global_step=gan.epoch)
                syn_latent_data = gan.generate(n_syn_samples, noise_params)
                syn_data = ae.decode(syn_latent_data)
                np.savez(osp.join(save_dir, 'epoch_' + str(epoch)), syn_data)
                for k in range(3):
                    Point_Cloud(syn_data[k]).plot(in_u_sphere=True)

            train_stats.append((epoch,) + loss)
        log_file.close()
        
    if do_plotting:
        x = range(len(train_stats))
        d_loss = [t[1] for t in train_stats]
        g_loss = [t[2] for t in train_stats]
        plt.plot(x, d_loss, '--')
        plt.plot(x, g_loss)
        plt.title('Latent GAN training. (%s, %s, %s-bneck)' %(class_name, ae_loss, ae.bottleneck_size))
        plt.legend(['Discriminator', 'Generator'], loc=0)

        plt.tick_params(axis='x', which='both', bottom='off', top='off')
        plt.tick_params(axis='y', which='both', left='off', right='off')

        plt.xlabel('Epochs.') 
        plt.ylabel('Loss.')

        plt.savefig(osp.join(train_dir, 'training_curve.png'))